# CIFAR10 by TensorFlow on SageMaker

<small>Copyright © 2018 by Arata Furukawa. (http://ornew.net)</small>

<small style="font-size:0.5em;">この資料は「機械学習アプリを「賢く」作る：Amazon SageMakerクラウド・ハンズオン」のために作成されたものです。ノートブックをダウンロードし、持ち帰って自由に実行・加工していただいて構いませんが、解説文の外部への公開・転載はご遠慮ください。</small>

---

[CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html)という分類問題を解いてみましょう。CIFAR10は以下の10種類の画像を分類する問題です。

1. 飛行機
2. 自動車
3. 鳥
4. 猫
5. 鹿
6. 犬
7. カエル
8. 馬
9. 船
10. トラック

In [ ]:
output_path = 's3://marukawa0224-XX/tensorflow'
base_job_name = 'marukawa0224-XX-cifar10-job'
print('output_path:\n\t{}'.format(output_path))
print('base_job_name:\n\t{}'.format(base_job_name))

In [ ]:
%matplotlib inline
import os
import uuid
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import sagemaker
from sagemaker import get_execution_role

session = sagemaker.Session()
role = sagemaker.get_execution_role()

実際はデータのダウンロードと変換とアップロードが必要ですが、MNISTと違いそこそこ時間がかかるため、ハンズオンでは事前にS3上にアップロードしたものをご利用ください。(持ち帰ってご自身で実行される場合は以下のプログラムを参考にしてください。)

```python
def convert_tfrecord(filename, images, labels):
    options = tf.python_io.TFRecordOptions(tf.python_io.TFRecordCompressionType.GZIP)
    with tf.python_io.TFRecordWriter(filename, options) as tfrecord:
        for image, label in zip(images, labels):
            example = tf.train.Example(features=tf.train.Features(feature={
                'image' : tf.train.Feature(
                    float_list=tf.train.FloatList(
                        value=image.flatten() / 255.)),
                'label' : tf.train.Feature(
                    int64_list=tf.train.Int64List(value=label)),
            }))
            tfrecord.write(example.SerializeToString())

train, test = tf.keras.datasets.cifar10.load_data()
data_dir = 'data/cifar10'
tf.gfile.MakeDirs(data_dir)
convert_tfrecord(os.path.join(data_dir,'train.tfr'), train[0], train[1])
convert_tfrecord(os.path.join(data_dir,'test.tfr'), test[0], test[1])

data_dir = session.upload_data(data_dir, session.default_bucket(), data_dir)
```

In [ ]:
data_dir = 's3://sagemaker-seminar-data/cifar10'

プログラムの説明の前に、学習を実行しましょう。しばらく時間がかかります。

job_nameやendpoint_nameは、もし途中で誤って中断してしまった場合や、あとから再度アタッチするのに必要な情報ですので、表示を残しておきましょう。

In [ ]:
hyperparameters = {
    'save_summary_steps'       : 10,
    'point_multiplier'         : 1,
    'depth_multiplier'         : 1,
    'learning_rate'            : 0.5,
    'dropout_rate'             : 0.5,
    'batch_size'               : 512,
    'shuffle_buffer_size'      : 1024,
    'weight_decay'             : 2e-4,
    'tfrecord_compression_type': 'GZIP',
    # tfrecord_compression_typeは、convert_tfrecordの時に
    # 指定した圧縮オプションと一致する必要があります
    # 事前アップロードしたものはGZIPです
}

from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(
    entry_point='./code/cifar10.py',
    hyperparameters=hyperparameters,
    role=role,
    output_path=output_path,
    code_location=output_path,
    training_steps=10000,
    evaluation_steps=1000,
    base_job_name=base_job_name,
    
    #########################################################
    # ハンズオンでは以下の設定を絶対に変えないでください！！！
    # 他の方が実行できなくなったり、高額な課金が発生することがあります。
    # 異常な課金等を確認した場合、請求額を追加徴収させて頂きます。
    #########################################################
    train_instance_count=1,
    train_instance_type='ml.p2.xlarge')

`run_tensorboard_locally`を指定するとTensorBoardも起動します。

<a href="/proxy/6006/" target="_blank">新しいタブでTensorBoardを開く</a>(下のセルのfitを実行してからアクセスしてください)

In [ ]:
%%time
estimator.fit(data_dir, run_tensorboard_locally=True)

## 中級テクニックの解説

ここから、今学習を実行しているモデルの解説をします。もし学習が早く終わったら、後ろの方の「デプロイ」を実行しておいても構いません。

### MobileNet

事前に用意したモデルでは、Googleの**MobileNet**というネットワーク構造を使っています。

最先端のモデルには若干精度で劣りますが、GoogleNetやVGG16といった有名なネットワークよりも高精度のベンチマークを出しています。

MobileNetの特徴として、

- 精度に対してパラメータ数が少ない
- パラメータ数が少ないので、計算負荷やモデルサイズが小さい
- 近年高精度を出している複雑なハイウェイ構造を持っていないため、とてもシンプルで理解しやすい

といった事が挙げられます。負荷が小さいため、名前の通りモバイル環境でも実行可能なモデルとして注目されています。

MobileNetには、ディープラーニングによる画像認識における重要なテクニックが幾つも盛り込まれています。いくつかの重要なテクニックについてピックアップして解説をします。

### MobileNetの構造

MobileNetを簡単に実装すると、以下のようなプログラムになります。

```python
# 畳み込みのブロック
def conv_block(x, filters, strides):
    x = tf.layers.conv2d(x, filters, 3, strides, 'same', use_bias=False)
    x = tf.layers.batch_normalization(x, training=is_training)
    x = tf.nn.relu(x)
    return x

# 分離型畳み込みのブロック
def sepconv_block(x, filters, strides):
    x = tf.layers.separable_conv2d(x, filters, 3, strides, 'same', use_bias=False)
    x = tf.layers.batch_normalization(x, training=is_training)
    x = tf.nn.relu(x)
    return x

with tf.variable_scope('mobilenet'):
    x =    conv_block(x, 32  , 2, '32-2')
    x = sepconv_block(x, 64  , 1, '64-1')
    x = sepconv_block(x, 128 , 2, '128-2')
    x = sepconv_block(x, 128 , 1, '128-1')
    x = sepconv_block(x, 256 , 2, '256-2')
    x = sepconv_block(x, 256 , 1, '256-1')
    x = sepconv_block(x, 512 , 2, '512-2')
    x = sepconv_block(x, 512 , 1, '512-1-0')
    x = sepconv_block(x, 512 , 1, '512-1-1')
    x = sepconv_block(x, 512 , 1, '512-1-2')
    x = sepconv_block(x, 512 , 1, '512-1-3')
    x = sepconv_block(x, 512 , 1, '512-1-4')
    x = sepconv_block(x, 1024, 2, '1024-2')
    x = sepconv_block(x, 1024, 1, '1024-1')
```

畳み込みのブロックが1個、その後に分離型畳み込みのブロックが13個繋がったモデルです。

各ブロックは、**入力 → (分離型)畳み込み → バッチ正規化 → ReLU(活性化関数) → 出力**という流れになっています。

モデル構造については、[TensorBoard](/proxy/6006/#graphs)も参考にしてください。

### 畳み込み層

MNISTの例では、$xW + b$という形の密結合（Densely Connected, 総結合=Fully Connectedとも）層を使いました。畳み込み層では、$W$と$x$について行列積の代わりに畳み込みという計算を行います。

畳み込み演算は、ディープラーニング以前から信号処理や画像処理で用いられていた計算で、画像の場合はエッジ抽出などの特徴抽出を行う処理として活用されていました。

![](https://i.imgur.com/6yBG7LC.png)
（図）簡略化のためにチャネル方向の次元を省略しているので注意。それぞれのサイズや数もあくまで一例です。

局所的な空間情報に基づく特徴の抽出が行えるため、画像認識におけるニューラルネットワークでは必ずと言っていいほど使われています。密結合に比べて計算量は多くなりますが、パラメータ数は少なくなります。

### 分離型畳み込み層（Separable Convolutional Layer）

通常の畳み込みをDepthwise（深度毎）とPointwise（位置毎）の2回の畳み込みに分離したものです。カーネルパラメータを低次元の2つのカーネルパラメータで近似することで、精度をある程度維持しながらパラメータ数を減らす事ができます。使う側としては通常の畳み込み層と同じように使えます。

### バッチ正規化（Batch Normalization）

![](https://i.imgur.com/gJ9Tg3k.png)

（図）入力データはミニバッチ。バッチ方向のデータ（青く塗りつぶされた部分）が、平均0、分散1となるように正規化を行う。これを位置ごとに行う。

MNISTの例では、一度の学習につき複数枚の入力を同時に行いました。入力のまとまりをバッチと呼びます。厳密には、全てのデータセットのまとまりをバッチといい、そこからランダムに選択した複数の入力をミニバッチと言いますが、まとめてバッチと呼称することもあります。

バッチ正規化では、位置毎にバッチ方向で入力データの正規化を行います。（図を参照）

内部共変量シフトが抑制されることで学習が安定します。学習係数を高く設定することができるようになり、収束速度が高速化します。正則化の効果もあり、勾配が爆発したり消失する現象を抑え、より層の深いネットワークの学習を可能にします。また、パラメータの初期値の影響を小さくしたり、データのばらつきによる過学習を抑える効果もあります。

共変量シフトとは、例えばデータのばらつきなどが原因で、関数の想定する入力分布と実際の入力分布が変わってしまう現象のことです。多層のニューラルネットワークの内部では中間層の出力が次の層の入力となりますが、この入力も同じように共変量シフトが発生します。これを内部共変量シフトと呼び、深いネットワークの学習に大きな悪影響を与えることが知られています。

### ReLU（Rectified Linear Unit）

ReLUは整流器と呼ばれる関数です。定義はシンプルで、$\max(x, 0)$です。一般的にはランプ関数と呼ばれます。入力が正数であれば入力をそのまま出力し、入力が負数であれば0を出力します。

ディープニューラルネットワークは層を広く深くすることで十分な表現力を獲得したため、活性化関数の非線形性が精度に与える影響は非常に小さくなりました。むしろ、活性化関数の影響で誤差逆伝搬中に勾配が非常に小さくなり学習が困難になる勾配消失の問題が発生します。区分線形関数であるReLUは、非線形関数でありながら微分値は1か0のいずれかであるため、誤差逆伝搬中に勾配の消失を引き起こさずに、深いネットワークの学習を可能にします。ReLUは微分不可能な点を持ちますが、計算上は劣微分で処理します。勾配計算が単純で高速であるのも特徴です。派生した関数も多く、上限値が設定されたもの、負の区間における勾配を変更したもの、パラメトリックなもの、微分可能に拡張したものなどがあります。深いネットワークにおけるデファクトスタンダードと言える重要な活性化関数です。

### 読み出し層の構造

MobileNetによってフィーチャを変換したのち、そこから目的となる10個のラベルの確率分布に変換するための読み出し層を定義します。

```python
# 読み出し層
with tf.variable_scope('readout_layer'):
    x = tf.reduce_mean(x, [1, 2], keep_dims=True, name='global_average_pooling')
    x = tf.layers.dropout(x, rate=0.5, training=is_training)
    x = tf.layers.conv2d(x, CATEGORY_NUM, 1)
    x = tf.squeeze(x, [1,2])

# 出力
logits = tf.identity(x, name='logits')
probabilities = tf.nn.softmax(logits, name='probabilities')
```

総平均プーリング層、ドロップアウト層、畳み込み層を経て、最後にSoftmax関数です。

### 総平均プーリング層（Global Average Pooling Layer）

平均プーリングとは、一定の範囲をその平均値に集約（プーリング）する操作です。総平均プーリングでは、入力レイヤー全体（今回の画像で言えば、高さと幅の軸で構成される面）単位で平均を取るプーリングを行います。畳み込みニューラルネットワークは、最後の読み出し層のパラメータ数がとても大きくなる傾向があります。読み出し層の前に適用することで読み出し層の重みパラメータ数を抑えることができます。このような一般的な畳み込みネットワークの読み出し層の前で総平均プーリングをしても精度に大きな影響はありません。

### ドロップアウト層（Dropout Layer）

学習の際に、ランダムにパラメータをドロップアウトする（存在しないものとして扱う）ことで、データの偏りによるモデルの過学習を抑えます。ただし、学習速度が遅くなる欠点があります。

## デプロイ

In [ ]:
api = estimator.deploy(
    #########################################################
    # ハンズオンでは以下の設定を絶対に変えないでください！！！
    # 他の方が実行できなくなったり、高額な課金が発生することがあります。
    # 異常な課金があった場合、請求額を追加徴収させて頂きます。
    #########################################################
    initial_instance_count=1,
    instance_type='ml.t2.medium')
print('Endpoint:\n\t{}'.format(api.endpoint))

In [ ]:
from sagemaker.tensorflow.predictor import tf_serializer, tf_deserializer
predictor = sagemaker.RealTimePredictor(endpoint=api.endpoint,
                              deserializer=tf_deserializer, 
                              serializer=tf_serializer,
                              content_type='application/octet-stream')

In [ ]:
from sagemaker.tensorflow.tensorflow_serving.apis import predict_pb2
def create_request(data):
    tensor_proto = tf.make_tensor_proto(
        values=data, shape=[1, 32, 32, 3], dtype=tf.float32)
    request = predict_pb2.PredictRequest()
    request.model_spec.name = 'generic_model'
    request.model_spec.signature_name = 'predictions'
    request.inputs['images'].CopyFrom(tensor_proto)
    return request

labels = [
    '飛行機',
    '自動車',
    '鳥',
    '猫',
    '鹿',
    '犬',
    'カエル',
    '馬',
    '船',
    'トラック',
]

def predict(image, label):
    result = predictor.predict(create_request(image.tolist()))
    plt.bar(np.arange(10), result.outputs['probabilities'].float_val)
    plt.ylim(ymax=1)
    plt.show()
    predict = result.outputs['classes'].int64_val[0]
    print('答え={} 予測={}'.format(labels[label], labels[predict]))
    plt.imshow(image)
    plt.show()

In [ ]:
import json
test_data = json.load(open('cifar10.test.json'))

for i in xrange(len(test_data['labels'])):
    image = test_data['data'][i]
    label = test_data['labels'][i]
    predict(np.asarray(image).astype(float) / 255., label)